# Pipeline
La pipeline fait référence à un ensemble d'étapes ou de processus qui sont exécutés séquentiellement pour traiter les données d'entrée, extraire des caractéristiques, entraîner le modèle et prédire les résultats.

In [38]:
import pandas as pd
import numpy as np
import os
from copy import deepcopy

# Global variables
GENERAL_DATA_PATH = "./data/general_data.csv"
EMPLOYEE_SURVEY_DATA_PATH = "./data/employee_survey_data.csv"
MANAGER_SURVEY_DATA_PATH = "./data/manager_survey_data.csv"
IN_TIME_DATA_PATH = "./data/in_out_time/in_time.csv"
OUT_TIME_DATA_PATH = "./data/in_out_time/out_time.csv"

## Acquisition des données
Collecter des données brutes à partir de diverses sources, telles que des capteurs, des fichiers, des bases de données, etc.

In [39]:
def load_data(data_path):
    csv_path = os.path.join(data_path)
    return pd.read_csv(csv_path, sep=',')

def agregate_dataframes(list_df, on_column, how):
    df_master = list_df[0]
    for df in list_df[1:]:
        df_master = df_master.merge(df,
                       on = on_column, 
                       how = how)
    return df_master

# Load data
df_general = load_data(GENERAL_DATA_PATH)
df_employee_survey = load_data(EMPLOYEE_SURVEY_DATA_PATH)
df_manager_survey = load_data(MANAGER_SURVEY_DATA_PATH)
df_in_time = load_data(IN_TIME_DATA_PATH)
df_out_time = load_data(OUT_TIME_DATA_PATH)
# Merge dataframes
df_total = agregate_dataframes(
    [df_general, df_employee_survey, df_manager_survey],
    "EmployeeID",
    "outer"
)

# Display a sample of 5 rows
df_total.sample(5)

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
535,28,No,Travel_Rarely,Human Resources,1,1,Human Resources,1,536,Male,...,4.0,2,4,2,2,1.0,3.0,2.0,3,3
153,60,No,Travel_Rarely,Research & Development,10,3,Life Sciences,1,154,Female,...,33.0,4,29,11,10,1.0,1.0,1.0,3,3
2369,35,No,Non-Travel,Research & Development,29,3,Medical,1,2370,Female,...,15.0,3,14,2,9,3.0,3.0,3.0,3,3
799,29,Yes,Travel_Frequently,Research & Development,28,3,Medical,1,800,Female,...,7.0,1,5,0,0,1.0,2.0,1.0,3,3
1729,25,No,Travel_Rarely,Research & Development,11,3,Medical,1,1730,Male,...,4.0,3,3,1,2,4.0,2.0,3.0,3,3


## Prétraitement des données
Nettoyer, normaliser et préparer les données pour l'analyse ultérieure.

### Nettoyage des données
Une étude préablable a permis de déterminer quels champs n'apportaient pas d'informations utiles à la prédiction. De même pour les champs non éthiques.

In [40]:
fields_not_useful = [
    "Over18",  # We have age
    "EducationField",  # We have Education
    "EmployeeCount",  # All at 1
    "StockOptionLevel",  # Not relevant
    "EmployeeID",  # Can be found in "index + 1"
]
fields_not_ethical = [
    "Gender",
    "MaritalStatus",
]

# Datframe with only useful fields
df_useful = deepcopy(df_total)
df_useful.drop(
    fields_not_useful,
    axis=1,
    inplace=True
)
# Datframe with useful and ethical fields
df_ethical = deepcopy(df_total)
df_ethical.drop(
    fields_not_useful + fields_not_ethical,
    axis=1,
    inplace=True
)

# Delete to free memory
del df_total
del fields_not_useful
del fields_not_ethical

### Normalisation des données

In [41]:
from sklearn.preprocessing import LabelEncoder

# Transform categorical variables into numerical
le = LabelEncoder()
for col in df_useful.columns:
    if df_useful[col].dtype == "object":
        df_useful[col] = le.fit_transform(df_useful[col])
for col in df_ethical.columns:
    if df_ethical[col].dtype == "object":
        df_ethical[col] = le.fit_transform(df_ethical[col])
# Display a sample of 5 rows
df_ethical.sample(5)

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,JobLevel,JobRole,MonthlyIncome,NumCompaniesWorked,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
1125,36,0,0,1,1,3,2,6,28380,1.0,...,5.0,5,5,0,2,1.0,2.0,3.0,1,3
3815,43,0,2,1,1,1,1,3,46780,5.0,...,8.0,0,6,0,2,4.0,2.0,3.0,4,4
3092,30,0,2,2,25,3,1,7,24060,2.0,...,11.0,5,5,0,2,3.0,4.0,3.0,3,4
699,40,0,0,1,1,1,1,2,42860,9.0,...,6.0,6,4,0,0,3.0,3.0,3.0,3,3
4008,32,1,2,1,1,1,2,2,27410,7.0,...,7.0,3,2,2,2,3.0,2.0,2.0,3,3


## Extraction de caractéristiques
Extraire les caractéristiques pertinentes des données brutes pour les utiliser dans l'apprentissage automatique.

## Sélection des modèles
Sélectionner le modèle d'apprentissage automatique le plus approprié pour le problème spécifique que l'on cherche à résoudre.

## Entraînement du modèle
Entraîner le modèle sélectionné sur les données d'entraînement en utilisant des algorithmes d'apprentissage automatique appropriés.

## Évaluation du modèle
Evaluer les performances du modèle sur des données de test pour mesurer sa précision, sa fiabilité et sa robustesse.

## Mise en production du modèle
Intégrer le modèle entraîné dans une application en temps réel pour effectuer des prédictions sur de nouvelles données.